<img src="images/econ151.png" width="200" />

<h1>ECON 151 Class 5</h1>

In this hands-on, we will step through an example of loading up and recoding a data extract from the expansive Integrated Public Use Microdata Series ([IPUMS](https://www.ipums.org)) database.

Learning objectives:

1. More experience with OLS
2. Better formatting, with `options(scipen=999)` and `table()` inside a `data.frame`
3. Recoding variables using `ifelse()` and `case_when()` inside the `dplyr` library
4. Estimation with survey weights

In [ ]:
library(ipumsr)
library(dplyr)
library(tidyverse)
library(haven)

It turns out that this handy command stops __R__ from defaulting to scientific notation.

In [ ]:
options(scipen=999)

<h2>Using IPUMS</h2>

IPUMS data must be downloaded from their website using an established user account. (You can also analyze data online, but I don't recommend that option.) Like with almost everything in __R__, there are at least 3 ways of working with IPUMS data once you've downloaded your extract from IPUMS and either uploaded it to datahub or installed your own (free) local copy of [RStudio desktop](https://www.rstudio.com/products/rstudio/) to examine the data.

There are at least three methods:
1. Download .DAT and .XML files from IPUMS and use `ipumsr`
2. Download Stata .DTA files from IPUMS and use `read_dta()` from `haven`
3. Download .CSV files from IPUMS and use `read.csv()` in base __R__

In this notebook, we'll look at the first way: using the `ipumsr` library functions below to read in the data from a .DAT and .XML (a.k.a. DDI) file. To pull this off, you need to have the .DAT and .XML files downloaded and placed into your working directory.

The following commands pull in my CPS extract `cps_00028`, which draws these variables from the January 2018 wave (job tenure supplement) of the Current Population Survey on IPUMS:

````{verbatim}
YEAR            Survey year
SERIAL          Household serial number
MONTH           Month
HWTFINL         Household weight, Basic Monthly
CPSID           CPSID, household record
STATEFIP        State (FIPS code)
METRO           Metropolitan central city status
PERNUM          Person number in sample unit
WTFINL          Final Basic Weight
CPSIDP          CPSID, person record
AGE             Age
SEX             Sex
RACE            Race
MARST           Marital status
VETSTAT         Veteran status
HISPAN          Hispanic origin
EMPSTAT         Employment status
OCC             Occupation
IND             Industry
CLASSWKR        Class of worker
AHRSWORKT       Hours worked last week
WKSTAT          Full or part time status
EDUC            Educational attainment recode
EDCYC           Years of college credit completed
EDDIPGED        High school or GED
EDHGCGED        Highest grade completed before receiving GED
JTYEARS         Length of time worked at current job in years
JTYEARAGO       Employment one year ago
JTSAME          Doing the same work a year ago
JTYPE           Type of business or organization in which you worked last year
JTCLASS         Class of worker, Job Tenure Supplement
JTIND           Industry one year ago
JTOCC           Occupation one year ago
JTSUPPWT        Job tenure supplement weight
EARNWEEK        Weekly earnings
````

In [ ]:
cps_data <- ipumsr::read_ipums_micro(
  ddi = "cps_00028.xml",
  data = "cps_00028.dat.gz"
)

In [ ]:
# Here is another version of calls that successfully load the data, 
# here drawing from the unzipped data file cps_00028.dat instead of the gzipped compressed file

#cps_ddi <- read_ipums_ddi("cps_00028.xml")
#cps_data <- read_ipums_micro(cps_ddi, verbose = FALSE)

In [ ]:
head(cps_data)

Let's clean up the data. But what variables need cleaning? You could browse the documentation for each one, or you could look at a quick `summary()` for suspect variables. Seasoned users of IPUMS will know that earnings and income contain real data <i>and codes for missing data</i> as explained for `EARNWEEK` [here for example](https://cps.ipums.org/cps-action/variables/EARNWEEK#codes_section).

In [ ]:
summary(cps_data$EARNWEEK)

Let's recode to drop those missing to NA's.

In [ ]:
cps_data <- mutate(cps_data, earnweek0 = ifelse(EARNWEEK < 9999.99, EARNWEEK, NA))

In [ ]:
cps_data <- mutate(cps_data, logearnweek = log(earnweek0))

A standard trick: when gender identity is binary in the data, it's usually coded for female == 2; subtract 1 to get an indicator for female identity.

In [ ]:
cps_data <- mutate(cps_data, female = SEX - 1)

The job tenure variable also has NA's coded as numbers, so let's drop those:

In [ ]:
cps_data <- mutate(cps_data, jtyears = ifelse(JTYEARS < 99.98, JTYEARS, NA))

The `HISPAN` variable takes on many values, [as shown here in the documentation](https://cps.ipums.org/cps-action/variables/HISPAN#codes_section). None missing, so a useful way forward is to assign 0 for `HISPAN` == 0 and 1 for everything else, which code different Hispanic identities. (Presumably Brazilians in the dataset are coded as `HISPAN` == 612 for "South American," but that's an open question.)

In [ ]:
cps_data <- mutate(cps_data, hispanic = ifelse(HISPAN == 0, 0, 1))

Here is my preferred quadrichotomous measure of race/ethnicity for the U.S. in 2022: 
1. Non-Hispanic white
2. Non-Hispanic Black
3. Hispanic, any race
4. Non-Hispanic other race 

In [ ]:
cps_data <- mutate(cps_data, raceth = ifelse(RACE == 100 & hispanic == 0, 1,
                                            ifelse(RACE == 200 & hispanic == 0, 2,
                                                   ifelse(hispanic == 1, 3, 4)))
                   )

Marital status is measured in `MARST`, and codes 1 and 2 mean the respondent is married, spouse present or absent.

In [ ]:
cps_data <- mutate(cps_data, married = ifelse(MARST == 1 | MARST == 2, 1, 0))

Let's look at the variable `EDUC`, which codes educational attainment. This code creates a more legible table of tabulated counts per value:

In [ ]:
temp = data.frame(table(cps_data$EDUC))
temp

These definitely aren't years of education. As [shown in the documentation](https://cps.ipums.org/cps-action/variables/EDUC#codes_section), this is what we're looking at:

````{verbatim}
*                           NIU or blank |    1
*                      None or preschool |    2
*                   Grades 1, 2, 3, or 4 |   10
*                          Grades 5 or 6 |   20
*                          Grades 7 or 8 |   30
*                                Grade 9 |   40
*                               Grade 10 |   50
*                               Grade 11 |   60
*                 12th grade, no diploma |   71
*      High school diploma or equivalent |   73
*             Some college but no degree |   81
*Associate's degree, occupational/vocati |   91
*   Associate's degree, academic program |   92
*                      Bachelor's degree |  111
*                        Master's degree |  123
*             Professional school degree |  124
*                       Doctorate degree |  125
````

This is an elegant way of recoding `EDUC` into something that economists think is useful: years of education attained. Here, we use `case_when()`, which is part of the `dpylr` package. Special thanks to UC Berkeley Demography Ph.D. student Casey Breen for this code.

The question of exactly how to map attainment categories to years of attainment is a complicated one. I've tried to adapt what [David Jaeger (2003) suggests](https://www.sciencedirect.com/science/article/pii/S0165176502002598).

(Another way of proceeding altogether would be to generate indicator variables for levels of attainment.)

In [ ]:
cps_data <- cps_data %>%
mutate(edyrs = case_when(
    EDUC == 2 ~ 0,
    EDUC == 10 ~ 2.5,
    EDUC == 20 ~ 5.5,
    EDUC == 30 ~ 7.5,
    EDUC == 40 ~ 9,
    EDUC == 50 ~ 10,
    EDUC == 60 ~ 11,
    EDUC == 71 ~ 12,
    EDUC == 73 ~ 12,
    EDUC == 81 ~ 14,
    EDUC == 91 ~ 14,
    EDUC == 92 ~ 14,
    EDUC == 111 ~ 16,
    EDUC == 123 ~ 18,
    EDUC == 124 ~ 18,
    EDUC == 125 ~ 18
    ))

Now let's look at what's here:

In [ ]:
temp = data.frame(table(cps_data$edyrs))
temp

Here is another bunch of code that is far uglier but produces the same thing, using a ton of nested `ifelse()` calls.

In [ ]:
#cps_data <- mutate(cps_data, edyrs = ifelse(EDUC == 2, 0,
#                                            ifelse(EDUC == 10, 2.5,
#                                                  ifelse(EDUC == 20, 5.5,
#                                                        ifelse(EDUC == 30, 7.5,
#                                                              ifelse(EDUC == 40, 9,
#                                                                    ifelse(EDUC == 50, 10,
#                                                                          ifelse(EDUC == 60, 11,
#                                                                                ifelse(EDUC == 71, 12,
#                                                                                      ifelse(EDUC == 73, 12,
#                                                                                            ifelse(EDUC == 81, 14,
#                                                                                                  ifelse(EDUC == 91, 14,
#                                                                                                        ifelse(EDUC == 92, 14,
#                                                                                                              ifelse(EDUC == 111, 16,
#                                                                                                                    ifelse(EDUC == 123, 18,
#                                                                                                                          ifelse(EDUC == 124, 18,
#                                                                                                                                ifelse(EDUC == 125, 18, NA)
#                                                                                                                                ))))))))))))))
#                                            )
#                   )

Finally, here are a few more recodings. Recall our earlier discussion that labor economists like to measure <i>experience</i> as years of age minus years of education.

In [ ]:
cps_data <- mutate(cps_data, exper = AGE - edyrs)

In [ ]:
cps_data <- mutate(cps_data, expersq = exper^2)

With all of these recodings, we are finally ready to go. Let's run this model of log earnings, where the treatment variables of interest are a binary measure of female gender identity and years of education, and we have a set of controls that includes race/ethnicity, years of experience, years of experience squared, and also years of job tenure:

$$
\ln Y_i = \alpha + \beta^f \ female_i + \beta^e \ educ_i + B \ controls_i + \epsilon_i
$$

Let's also restrict the sample to workers aged 25-64.

In [ ]:
cps_reg1 <- lm(logearnweek ~ female + edyrs + factor(raceth) + exper + expersq + jtyears, 
               data = subset(cps_data, AGE >= 25 & AGE <= 64)
               )
summary(cps_reg1)

Do you remember the tricks about quadratics? What is the $x$-coordinate of the vertex? If the (partial) function we have is:

$$
\ln Y_i = a \cdot exper_i^2 + b \cdot exper_i + c
$$

then the vertex falls where $exper_i = -b \ / \ 2a$. You can also find this by differentiating the formula above and setting the derivative to zero, which reveals that $2a \cdot exper_i + b = 0$, and solving for $exper_i$. 

Use this formula and plug in the coefficient on `exper` for $b$ and the coefficient on `expersq` for $a$ to find:

In [ ]:
0.02694388/(2*0.00045188)

In words, log earnings rise with experience up to about year 30. For a college graduate, that would be around age 52.

<hr>

What about marital status? We didn't include that, nor did we control for a bunch of other interesting variables in the data, like veteran status and other characteristics of job tenure. Let's try inserted `married` to see what we find:

In [ ]:
cps_reg2 <- lm(logearnweek ~ female + edyrs + factor(raceth) + exper + expersq + jtyears + married, 
               data = subset(cps_data, AGE >= 25 & AGE <= 64)
               )
summary(cps_reg2)

Describe what you see. Are these results fundamentally different or mostly the same as the results without controlling for `married()`?

<hr>

We have 2 sampling weights in the extract: `WTFINL` the final basic weight, and `JTSUPPWT` the job tenure supplement weight. If we use job tenure variables in a model, we should use the latter weight. Let's give it a try.

In [ ]:
cps_reg3 <- lm(logearnweek ~ female + edyrs + factor(raceth) + exper + expersq + jtyears + married, 
               data = subset(cps_data, AGE >= 25 & AGE <= 64),
               weight = JTSUPPWT
              )
summary(cps_reg3)

Describe what you see. Are these results fundamentally different or mostly the same as the results without sampling weights?

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>